Osnabrück University - Machine Learning (Summer Term 2016) - Prof. Dr.-Ing. G. Heidemann, Ulf Krumnack

# Exercise Sheet 08

## Introduction

This week's sheet should be solved and handed in before the end of **Sunday, June 12, 2016**. If you need help (and Google and other resources were not enough), feel free to contact your groups' designated tutor or whomever of us you run into first. Please upload your results to your group's Stud.IP folder.

## Assignment 1: Multilayer Perceptron (MLP) [10 Points]

Last week you implemented a simple perceptron. We discussed that one can use multiple perceptrons to build a network. This week you will build your own MLP. Again the following code cells are just a guideline. If you feel like it, just follow the algorithm steps and implement the MLP yourself.

### Implementation

In the following you will be guided through implementing an MLP step by step. Instead of sticking to this guide, you are free to take a complete custom approach instead if you wish.

We will take a bottom-up approach: Starting from an individual **perceptron** (or neruon), we will derive a **layer of perceptrons** and end up with a **multilayer perceptron** (or neural network). Each step will be implemented as its own python *class*. Such a class defines a type of element which can be instantiated multiple times. You can think of the relation between such instances and their designated classes as individuals of a specific population (e.g. Bernard and Bianca are both individuals of the population mice). On the classes methods are defined, which can be used after having instanciated an instance in order to manipulate it or to make it perform a specific action – again, taking the population reference, each mouse of a mice population would for example have the method `eat_cheese`.

To guide you along, all required classes and functions are outlined in valid python code with extensive comments. You just need to fill in the gaps. Each comment describes the arguments the specific method accepts (`Args`) and the values it is expected to return (`Returns`).

### Perceptron
Simillar to last week you here need to implement a perceptron. But instead of applying it directly, we will do so in a class which is reusable for an theoretically infinite amount of perceptrons. For the perceptron you need to implement the following three things:

#### Weight initialization

The weights are initialized by sampling values from a standard normal distribution. There are as many weights as there are dimensions in the vector – and one more for the bias.

#### Forward-Propagation / Activation

Calculate the weighted sums of a neuron's inputs and apply it's activation function $\sigma$. The output vector $o$ of perceptron $j$ of layer $k$ given an input (the output of the previous layer) $x$ in a neural network is given by the following formula. Note: $N$ gives the number of values of a given vector, $w_{j,0}(k)$ specifies the bias of perceptron $j$ in layer $k$ and $w_{j,1...N(x)}(k)$ the other weights of perceptron $j$ in layer $k$.

$$\begin{align*}
  o_{k,j}(x) = \sigma\left(w_{j,0}(k)+\sum\limits_{i=1}^{N(x)} x_i w_{j,i}(k)\right)
\end{align*}$$
with 
$$\sigma(x) = \frac{1}{1 + \exp{(-x)}}$$

Think of weights $w(k)$ as a matrix being located inbetween the layer $k$ and the layer located *to its left* in the network. So values flowing from layer $k-1$ to layer $k$ are weighted by the values of $w(k)$.

#### Back-Propagation / Adaptation
The perceptron itself does not need to worry about calculating its $\Delta$ weight adaptation value. Instead, it is provided by the network. Still, the perceptron needs to adjust its weights by the specified delta weighted by a given learning rate:

$$\Delta w_{j,i}(k) = \epsilon\, \delta_j(k+1)\, o_i(k)$$

In [ ]:
import numpy as np
from scipy.special import expit as sigmoid
# sigmoid = lambda x: 1 / (1 + np.exp(-x))


class Perceptron:
    """Single neuron handling its own weights and bias."""

    def __init__(self, dim_in, act_func=sigmoid):
        """Initialize a new neuron with its weights and bias.

        Args:
            dim_in: Dimensionality of the data coming into
                this perceptron. In a network of perceptrons
                this basically represents the number of neurons
                in the layer before this neuron's layer. Used
                for generating the perceptron's weights vector,
                which not only includes one weight per input but
                also an additional bias weight.
            act_fun: Function to apply on activation.
        """
        self.act_func = act_func
        # self.weights = []  # TODO
        self.weights = np.random.normal(size=dim_in + 1)

    def activate(self, x):
        """Activate this neuron with a specific input.

        Calculate the weighted sum of inputs and apply the
        activation function.

        Args:
            x: Vector of input values.

        Returns:
            A real number representing the perceptron's
            activation after calculating the weighted sum
            of inputs and applying the perceptron's
            activation function.
        """
        # return self.act_func(0)  # TODO
        return self.act_func(self.weights @ np.append(1, x))

    def adapt(self, x, delta, rate=0.03):
        """Adapt this neuron's weights by a specific delta.

        Args:
            x: Vector of input values.
            delta: Weight adaptation delta value.
            rate: Learning rate.
        """
        # self.weights += 0  # TODO
        self.weights += rate * delta * np.append(1, x)


# TODO (ahoereth): Add asserts testing the perceptron.

### PerceptronLayer
A `PerceptronLayer` is a combination of multiple `Perceptron` instances. It mainly is concerened with passing input and delta values to its individual neurons. There is no math to be done here!

#### Initialization

When initializing a `PerceptronLayer` like this: `layer = PerceptronLayer(5, 3)`, the `__init__` function is called. It creates a list of `Perceptron`s: For each output dimension there must be one perceptron. Each of those perceptrons has the same input dimensions and activation functions as the perceptron layer.

#### Activation

During the activation step, the perceptron layer activates each of its perceptrons (loop over them) and stores the results in a new list. This list is `return`ed (this is important, as the MLP needs the values later to calculate the errors and updates).

#### Adaptation

To adapt its perceptrons, the perceptron layer adapts each with the corresponding delta. For this purpose, the MLP passes a list of input values and a list of deltas to the adaptation function. The inputs are passed to *all* perceptrons. The list of deltas is exactly as long as the list of perceptrons: The first delta is for the first perceptron, the second for the second, etc.

In [ ]:
class PerceptronLayer:
    """Layer of multiple neurons."""

    def __init__(self, dim_in, dim_out, act_func=sigmoid):
        """Initialize the layer as a list of individual neurons.

        A layer contains as many neurons as it has outputs, each
        neuron has as many input weights (+ bias) as the layer has inputs.
        
        Args:
            dim_in: Dimensionality of the expected input values,
                also the size of the previous layer of a neural network.
            dim_out: Dimensionality of the output, also the requested 
                amount of in this layer and the input dimension of the
                next layer.
            act_func: Activation function to use in each perceptron of
                this layer.
        """
        # self.perceptrons = []  # TODO
        self.perceptrons = [Perceptron(dim_in, act_func)
                            for _ in range(dim_out)]

    def activate(self, x):
        """Activate this layer by activating each individual neuron.
        
        Args:
            x: Vector of input values.
            
        Retuns:
            Vector (should be an np.array) of output values which can be used as input to
            another PerceptronLayer instance.
        """
        # return np.asarray([])  # TODO
        return np.asarray([p.activate(x) for p in self.perceptrons])

    def adapt(self, x, deltas, rate=0.03):
        """Adapt this layer by adapting each individual neuron.
        
        Args:
            x: Vector of input values.
            deltas: Vector of delta values.
            rate: Learning rate.
        """
        # pass  # TODO
        for perceptron, delta in zip(self.perceptrons, deltas):
            perceptron.adapt(x, delta, rate)

    def get_weights_matrix(self):
        """Helper function for getting this layer's weight matrix.
        
        Returns:
            Numpy array of all the weights for this perceptron layer.
        """
        return np.asarray([p.weights for p in self.perceptrons]).T


# TODO (ahoereth): Add asserts testing the perceptron layer.

### MultilayerPerceptron

#### Forward-Propagation / Activation
Propagate the input value $x$ through each layer of the network, employing the output of the previous layer as input to the next layer.

#### Back-Propagation / Adaptation
This is the most complex step of the problem. Split into 3 parts:

1. Compute individual output values for each layer – similar to the forward-propagation step above, but we need to keep track of the intermediate results. That means: Store the input as the first "output". Activate the first hidden layer using the *last* output and append its activation to the outputs. Continue in the same way: For the next hidden layer use the *last* output again as the new input, activate it, store the results. Do this until all layers are activated in order.

2. Calculate error signals $\delta_i$ for all layers $k$. Start by calculating the *network* error `e`, which is the difference between the output and the target `t`. Then, for all layers (starting from behind), calculate the deltas:

   $$\delta_i(k) = o_i(k)\ (1 - o_i(k))\ \sum\limits_{j=1}^{N(k+1)} w_{ji}(k+1,k)\delta_j(k+1)$$
   
    (*Hint*: For the last layer (i.e. the first you calculate the $\delta$ for) the sum in the formula above is your network error. For all preceding layers $k$ you need to recalculate `e` using the $\delta$ and weights of layer $k+1$. We already implemented a help function for you to access the weights of a specific layer. Check the `PerceptronLayer` if you did not find it yet.

3. After calculating the `e` for the next layer, adapt the weights the current layer by $\Delta$. Make sure that you use the non-updated weights for the calculations of the propagating $\delta$. 

$$\Delta(k+1) = \epsilon\, \delta_j(k+1)\, o_i(k)$$

In [ ]:
class MultilayerPerceptron:
    """Network of perceptrons, also a set of multiple perceptron layers."""

    def __init__(self, *layers):
        """Initialize a new network, madeup of individual PerceptronLayers.

        Args:
            *layers: Arbritrarily many PerceptronLayer instances.
        """
        self.layers = layers

    def activate(self, x):
        """Activate network and return the last layer's output.

        Args:
            x: Vector of input values.

        Returns:
            Vector of output values from the last layer of the network
            after propagating forward through the network.
        """
        for layer in self.layers:
            x = layer.activate(x)
        return x

    def adapt(self, x, t, rate=0.03):
        """Adapt the whole network given an input and expected output.

        Args:
            x: Vector of input values.
            t: Vector of target values (expected outputs).
            rate: Learning rate.
        """
        # Activate each layer and collect intermediate outputs.
        # outputs = []  # TODO
        outputs = [x]
        for layer in self.layers:
            outputs.append(layer.activate(outputs[-1]))

        # Calculate error between t and network output.
        # e = []  # TODO
        e = t - outputs[-1]

        # Backpropagate error through the network and adapt each layer.
        # TODO (ahoereth): Get rid of zip here because its 
        #     difficult to grasp not only for python beginners.
        layers = list(zip(self.layers, outputs[:-1], outputs[1:]))
        for layer, x, y in reversed(layers):
            delta = (y * (1 - y)) * e
            weights = layer.get_weights_matrix()
            e = (weights @ delta)[1:]
            layer.adapt(x, delta, rate)


# TODO (ahoereth): Add asserts testing the MLP.

### Classification

#### Problem Definition
Before we start, we need a problem to solve. In the following cell we first generate some three dimensional data (= $\text{input_dim}$) between 0 and 1 and label all data according to a binary classification: If the data is close to the center (radius < 2.5), it belongs to one class, if it is further away from the center it belongs to the other class.

In the cell below we visualize the data set.

In [ ]:
def uniform(a, b, n=1):
    """Returns n floats uniformly distributed between a and b."""
    return (b - a) * np.random.random_sample(n) + a

n = 1000
radius = 5
r = np.append(uniform(0, radius * .5, n // 2),
              uniform(radius * .7, radius, n // 2))
angle = uniform(0, 2*np.pi, n)
x = r * np.sin(angle) + uniform(-radius, radius, n)
y = r * np.cos(angle) + uniform(-radius, radius, n)
inputs = np.vstack((x, y)).T
targets = np.less(np.linalg.norm(inputs, axis=1), radius * .5)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize=(5,5))
plt.suptitle('Labeled Data')
plt.scatter(*inputs.T, 2, c=targets, cmap='RdYlBu');

### Model Design

In [ ]:
MLP = MultilayerPerceptron(
    PerceptronLayer(2, 4),
    PerceptronLayer(4, 2),
    PerceptronLayer(2, 1),
)

### Training

In [ ]:
import sys

EPOCHS = 100000

max_accuracy = 0

for epoch in range(0, EPOCHS + 1):
    s = np.random.randint(0, len(targets))
    MLP.adapt(inputs[s], targets[s])

    if epoch % 2500 == 0:
        outputs = np.squeeze([MLP.activate(x) for x in inputs])
        predictions = np.round(outputs)
        accuracy = np.sum(predictions == targets) / len(targets)
        max_accuracy = max_accuracy if accuracy < max_accuracy else accuracy
        sys.stdout.write('\rAccuracy at epoch {:6d}: {:.4f}%. Max accuracy: {:.4f}%'
                         .format(epoch, accuracy, max_accuracy))
        sys.stdout.flush()

### Evaluation

In [ ]:
plt.figure(figsize=(7,7))
plt.suptitle('Evaluation')
ax = plt.subplot(2,2,1)
ax.scatter(*inputs.T, 2, c=outputs, cmap='RdYlBu')
ax.set_title('Continious Classification')
ax = plt.subplot(2,2,2)
ax.set_title('Discretized Classification')
ax.scatter(*inputs.T, 2, c=np.round(outputs), cmap='RdYlBu')
ax = plt.subplot(2,2,3)
ax.set_title('Labels')
ax.scatter(*inputs.T, 2, c=targets, cmap='RdYlBu')
ax = plt.subplot(2,2,4)
ax.set_title('Wrong Classifications')
ax.scatter(*inputs.T, 2, c=(targets!=np.round(outputs)), cmap='OrRd');

## Assignment 2: MLP and RBFN [10 Points]

This exercise is aimed at deepening the understanding of Radial Basis Function Networks and how they relate to Multilayer Perceptrons. Not all of the answers can be found directly in the slides - so when answering the (more algorithmic) questions, first take a minute and think about how you would go about solving them and if nothing comes to mind search the internet for a little bit. If you are interested in a real life application of both algorithms and how they compare take a look at this paper: [Comparison between Multi-Layer Perceptron and Radial Basis Function Networks for Sediment Load Estimation in a Tropical Watershed](http://file.scirp.org/pdf/JWARP20121000014_80441700.pdf)

![Schematic of a RBFN](RBFN.png)

We have prepared a little example that shows how radial basis function approximation works in Python. This is not an example implementation of a RBFN but illustrates the work of the hidden neurons.

In [ ]:
%matplotlib inline

import numpy as np
from numpy.random import uniform

from scipy.interpolate import Rbf

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm


def func(x,y):
    '''
    This is the example function that should be fitted.
    Its shape could be described as two peaks close to
    each other - one going up, the other going down
    '''
    return (x + y) * np.exp(-4.0 * (x**2 + y**2))

# number of training points (you may try different values here)
training_size = 50

# sample 'training_size' data points from the input space [-1,1]x[-1,1] ...
x = uniform(-1.0, 1.0, size=training_size)
y = uniform(-1.0, 1.0, size=training_size)

# ... and compute function values for them.
fvals = func(x, y)

# get the aprroximation via RBF
new_func = Rbf(x, y, fvals)


# Plot both functions: 
# create a 100x100 grid of input values
x_grid, y_grid = np.mgrid[-1:1:100j, -1:1:100j]

plt.figure("Original Function")
# This plot represents the original function
f_orig = func(x_grid, y_grid)
plt.imshow(f_orig, extent=[-1,1,-1,1], cmap=plt.cm.jet)

plt.figure("RBF Result")
# This plots the approximation of the original function by the RBF
# if the plot looks strange try to run it again, the sampling
# in the beginning is random
f_new = new_func(x_grid, y_grid)
plt.imshow(f_new, extent=[-1,1,-1,1], cmap=plt.cm.jet)
plt.xlim(-1,1)
plt.ylim(-1,1)
# scatter the datapoints that have been used by the RBF
plt.scatter(x, y);

### Radial Basis Function Networks

#### What are radial basis functions?

Radial basis functions are all functions that fullfill the following criteria:

The value of the function for a certain point depends only on the distance of that point to the origin or some other fixed center point. In mathematical formulation that spells out to: 
$\phi (\mathbf {x} )=\phi (\|\mathbf {x} \|)$  or  $\phi (\mathbf {x} ,\mathbf {c} )=\phi (\|\mathbf {x} -\mathbf {c} \|)$. Notice that it is not necessary (but most common) to use the norm as the measure of distance.

#### What is the structure of a RBFN? You may also use the notion from the above included picture.

RBFN's are networks that contain only one hidden layer. The input is connected to all the hidden units. Each of the hidden units has a different radial basis function that is *sensitive* to ranges in the input domain. The output is then a linear combination of the outpus ot those functions.

#### How is a RBFN trained?

Note: all input data has to be normalized.

Training a RBFN is a two-step process. First the functions in the hidden layer are initialized. This can be either done by sampling from the input data or by first performing a k-means clustering, where k is the number of nodes that have to be initialzed.

The second step fits a linear model with coefficients $w_{i}$ to the hidden layer's outputs with respect to some objective function. The objective function depends on the task: it can be the least squares function, or the weights can be adapted by gradient descent.

### Comparison to the Multilayer Perceptron

#### What do both models have in common? Where do they differ?

|RBFN                 |MLP                  | 
|---------------------|---------------------|
| non-linear layered feedforward network|non-linear layered feedforward network| 
| hidden neurons use radial basis functions, output neurons use linear function| input, hidden and output-layer all use the same activation function| 
| universal approximator |   universal approximator |
| learning usually affects only one or some RBF | learning affects many weights throught the network|

#### How can classification in both networks be visualized?

![Classification](Solution_Classification.png)

#### When would you use a RBFN instead of a Multilayer Perceptron?

RBFNs are more robust to noise and should therefore be used when the data contains false-positives.